In [1]:
!pip install termcolor --break-system-packages
import json
import matplotlib.pyplot as plt
import numpy as np
import data_handling as dh
from data_gen import Datagen
from termcolor import colored
from scipy.stats import ttest_ind



types = {"tdlgm": "../tdlgm/results"}#, "dlgm": "../dlgm/results", "rnn": "../rnn/results"}

from IPython.display import HTML as html_print

def color_print(s, color='black'):
    display(html_print(("<text style=color:{}>{}</text>".format(color, s))))

color_print("Hello", "red")




def allowed(file):
    lowest = 1
    for i in range(len(file)):
        for k in range(len(file)):
            res = ttest_ind(file[i], file[k], equal_var = False).pvalue
            if res < lowest:
                lowest = res
    print(lowest)
    return lowest

def get_file(filename="entries"):
    l = None
    with open(filename, "r") as f:
        l = json.loads(f.read())
    return l

def draw_other(y,y_t, title, name=""):
    fig, ax = plt.subplots(2, layout="constrained")
    
    ax[0].plot( range(1,101) , y_t[:100])
    ax[0].plot( range(1,101) , y[:100])
    
    ax[1].plot( range(400,500) , y_t[400:])
    ax[1].plot(range(400,500), y[400:])
    
    for a in ax:
        a.set_xlabel("Timestep")
        a.set_ylabel("Fraction of GPU")
    
    fig.suptitle(title)
    if name != "":
        fig.savefig(name)
        plt.close("all")



Defaulting to user installation because normal site-packages is not writeable


/home/carladm/Documents/tDLGM/experiments/data_parsing/data_gen.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m = df.max()[0]
/home/carladm/Documents/tDLGM/experiments/data_parsing/data_gen.py:68: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m = df.max()[0]


In [2]:



rnn_files = dh.get_all_files(types["rnn"])

KeyError: 'rnn'

In [ ]:
from data_gen import Datagen
import matplotlib.pyplot as plt

gen = Datagen(None)

_, y_val, _ = gen.get_test_data(1)


def get_file(filename="entries"):
    l = None
    with open(filename, "r") as f:
        l = json.loads(f.read())
    return l

def draw_other(y,y_t, title, name=""):
    fig, ax = plt.subplots(2, layout="constrained")
    
    ax[0].plot( range(1,101) , y_t[:100])
    ax[0].plot( range(1,101) , y[:100])
    
    ax[1].plot( range(400,500) , y_t[400:])
    ax[1].plot(range(400,500), y[400:])
    
    for a in ax:
        a.set_xlabel("Timestep")
        a.set_ylabel("Fraction of GPU")
    
    fig.suptitle(title)
    if name != "":
        fig.savefig(name)
        plt.close("all")
    else:
        plt.show()
        plt.close()








results = []
for i in rnn_files:
    f = dh.get_file( types["rnn"] + "/"  + i)

    to_add = {}
    to_add["name"] = i
    to_add["entries"] = []
    values = []
    for p in f[0]["test_run"]:
        entry = {}
        entry["variance"] = p["variance"]
        entry["probability"] = p["probability"]
        arr = np.concatenate(p["diff"])
        values.append(arr)
        entry["mean"] = np.mean(arr)
        entry["var"] = np.var(arr)

        to_add["entries"].append(entry)

    p = allowed(values)
    print("*" * 20, "\n", i, "\n")
    color = "red"
    title="ERROR"
    if p < 0.9:
        color = "green"
        title = "ALLOWED"
    draw_other(f[0]["test_run"][0]["results"][0], y_val, title)
    color_print([k["var"] for k in to_add["entries"]], color)
    print( "*" * 20)
    to_add["p_score"] = p
    results.append(to_add)

    



In [ ]:

with open("output_rnn_prob.json", "w") as f:
    f.write(json.dumps(results))
